In [ ]:
'''
Condition GPT-3 a pre-specified number of randomized human generations
'''

import os
import openai
from dotenv import load_dotenv
import numpy as np 
import pandas as pd 
import random

load_dotenv()

openai.api_key = "" # todo: fill in with own key

random.seed(10)


In [ ]:
# load in human generations 
# for now, read directly from .txt files
exp_results_dir = "/Users/kcollins/language_and_structure_of_thoughts/plans/exp_results/"

seed_example_type = "random" 

data_file = "plans_per_goal.txt"

with open(f"{exp_results_dir}unconstrained_planning_pilot/{data_file}", "r") as f: 
    unconstrained_data = f.readlines()
    
with open(f"{exp_results_dir}constrained_single_objs/{data_file}", "r") as f: 
    constrained_single_data = f.readlines()

with open(f"{exp_results_dir}constrained_many_objs/{data_file}", "r") as f: 
    constrained_many_data = f.readlines()



In [ ]:
def is_goal(line): 
    # check if a line is a goal
    return line[:4] == "Goal"

def is_plan(line):
    # check if a line is the start of a plan
    return line[1:5] == "Plan"

def process_plan(plan):
    # remove starting "Plan: " and end "/n" 
    plan = plan[7:-1]
    # make sure that plan ends in a period 
    # remove all intermediate breaks -- replace with a period, or nothing if already exists a period
    plan = plan.replace(".<br />", ". ") # keep period
    plan = plan.replace(". <br />", ". ")
    plan = plan.replace("<br />", ". ") # add period 
    
    # for the final quote, make sure that there are not big gaps with extra spaces
    plan = plan.replace(".  \"", ".\"")
    if plan[-3:] == ". \"": 
        plan = plan[:-3] + ".\"" # remove trailing space before end
        
    # if no period at the end of the sentence, add it in
    # (for consistency w/ end token used w/ gpt-3)
    if plan[-2:] != ".\"": plan = plan[:-1] + ".\""
    
    return plan 
    
def get_plans_per_goal(data): 
    plans_per_goal = {}
    for i, line in enumerate(data): 
        if is_goal(line): 
            goal = line[:-1] # remove ending new line character
            # extract all plans for that goal
            plans = []
            for j, poss_plan in enumerate(data[i+1:]):
                if is_goal(poss_plan): break # on to a new goal --- save prior plans and move-on
                else: 
                    if is_plan(poss_plan): 
                        plan = process_plan(poss_plan)
#                       formatted_plan_str = f"Plan: \"{plan}\""

                        # note: manually inspect that they all are nested in quotes! 
                        formatted_plan_str = f"Plan: {plan}"

                        plans.append(formatted_plan_str)
            if seed_example_type == "best": # first plan loaded in is the best (by construction)
                plans_per_goal[goal] = {"plans": [plans[0]], "n_plans": len(plans)} 
            elif seed_example_type == "worst": # only take last, b/c sorted in reverse order of goodness
                plans_per_goal[goal] = {"plans": [plans[-1]], "n_plans": len(plans)}  
            else: plans_per_goal[goal] = {"plans": plans, "n_plans": len(plans)}
            
    return plans_per_goal

unconstrained_plans_per_goal = get_plans_per_goal(unconstrained_data)
constrained_single_plans_per_goal = get_plans_per_goal(constrained_single_data)
constrained_many_plans_per_goal = get_plans_per_goal(constrained_many_data)

In [ ]:
def sample_gpt3(prompt, n_rollouts=3, n_tokens=500, stop_token=".\n", temp=0.5): 
    # sample gpt-3
    response = openai.Completion.create(
      engine="davinci",
      prompt=prompt,
      temperature=temp,
      max_tokens=n_tokens,
      top_p=1,
      logprobs=5,
      frequency_penalty=0,
      presence_penalty=0,
    n= n_rollouts,
    stop=[stop_token]
    )
    return response


def create_inductive_prompt(human_goal_data, target_goal, n_examples=15): 
    
    # prohibit inclusion of target goal in seed prompts
    prompt_goals = np.random.choice(list(set(human_goal_data.keys()) - {target_goal}), n_examples, replace=False)

    seed_goals_and_plans = ""
    for goal in prompt_goals:
        plans = human_goal_data[goal]["plans"]
        seed_goals_and_plans += f"{goal}\n"
        
        if seed_example_type == "random": 
            # randomly select example to seed with
            plan = random.choice(plans)
        else: plan = plans[0] # b/c only one in list (either best or worst)
        seed_goals_and_plans += f"{plan}\n"
    query_goal = f"{target_goal}\nPlan:\"I would"

    
    return seed_goals_and_plans + query_goal

In [ ]:
# params to control gpt-3 generation 
temp=0.5
stop_token = ".\"\n"
n_tokens = 300
num_examples = 15 # number of goals to condition with

# define number of total roll-outs we want
draw_n_samples = 30

goal_type = "unconstrained"#"unconstrained"
save_dir = "./gpt3_generations/"

if not os.path.isdir(save_dir): os.makedirs(save_dir)

if goal_type == "unconstrained": 
    human_goal_data = unconstrained_plans_per_goal
elif goal_type == "constrained_single": 
    human_goal_data = constrained_single_plans_per_goal
elif goal_type == "constrained_many": 
    human_goal_data = constrained_many_plans_per_goal
    

# save for uploading to cognition.run
goals = []
plans = []
prompts = [] 
subj_ids = [] # for gpt-3 (always just add 'gpt-3')
goal_types = []

for idx, (target_goal, human_plan_data) in enumerate(human_goal_data.items()):
    
    goal_prefix = target_goal.split("Goal: ")[-1].split(",")[0].split(".")[0]
        
    print(target_goal)
        
    num_rerun = None # may need to rerun if rollout is incomplete
    
    while (num_rerun is None) or (num_rerun > 0):
        
        print("num rerun: ", num_rerun)
        
        if num_rerun is None: 
            num_rerun = 0 
            n_rollouts = draw_n_samples
        else:
            # number of samples to draw is the same as the number needed to be rerun
            n_rollouts = num_rerun
            num_rerun = 0
            
        # new prompt per rollout
        for _ in range(n_rollouts): 
            # sample a random set of k goals, that do not include the target goal 
            prompt = create_inductive_prompt(human_goal_data, target_goal, n_examples=num_examples)
            response = sample_gpt3(prompt, n_rollouts=1, 
                                   n_tokens=n_tokens, stop_token=stop_token, temp=temp)
            
            sampled_plan = response["choices"][0]["text"] # b/c only one rollout here
            
            # check if gpt-3 started to generate another goal/plan pair
            # if so, only keep part before this (note, this rarely happens - mainly in single case, if at all)
            if "\"\nGoal:" in sampled_plan: 
                sampled_plan = sampled_plan[:sampled_plan.rfind("\"\nGoal:")+1]
                 # only add period in this case, as moving on to new goal signals termination
                if sampled_plan[-1] != ".": sampled_plan += "."
                    
            # make sure plan ends on a period 
            if "." in sampled_plan: 
                sampled_plan = sampled_plan[:sampled_plan.rfind(".")+1]
                plans.append(f"I would{sampled_plan}")
                goals.append(target_goal)
                subj_ids.append("gpt-3")
                prompts.append(prompt)
                goal_types.append(goal_type)
            else: # need to resample
                num_rerun += 1

goal_plan_df = pd.DataFrame({"goal": goals, "plan": plans, "id": subj_ids, "prompt": prompts, "goal_type": goal_types})
goal_plan_df.to_csv(f"{save_dir}/max_inductive_{goal_type}_plans_full.csv")

In [ ]:
goal_plan_df = pd.DataFrame({"goal": goals, "plan": plans, "id": subj_ids, "prompt": prompts, "goal_type": goal_types})
goal_plan_df.to_csv(f"{save_dir}/max_inductive_{goal_type}_plans_full.csv")

In [ ]:
goals = [goal[6:] for goal in goals] # just remove starting 'Goal: '
goal_plan_df = pd.DataFrame({"goal": goals, "plan": plans, "id": subj_ids, "prompt": prompts, "goal_type": goal_types})

print(len(goal_plan_df))

In [ ]:
# batch s.t. plans are randomly put into 45 groups 

def compute_plan_batches(df, num_in_batch=45): 
    df = df.sample(frac=1).reset_index(drop=True)
    df["batch_idx"] = [idx // num_in_batch for idx in df.index]
    num_tot_batches = len(set(df["batch_idx"]))
    return df, num_tot_batches

batched_df, num_tot_batches = compute_plan_batches(goal_plan_df, num_in_batch=45)

batched_df.to_csv(f"{save_dir}/max_inductive_{goal_type}_plans_full.csv")